## Trabajo Practico Algebra Lineal Computacional

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math 
import sympy as sym
import scipy as la

## Ejercicio 13

In [2]:
# dado un 'L' y un 'b' resuelve Lx = b
def res_L(L, b):
    y = np.zeros(len(b))
    for i in range(len(b)):
        y[i] = ( b[i] - (L[i, :i] @ y[:i]) ) / L[i,i]
    return y


#Dado un 'U' y un 'y' resuelve Ux = y
def res_U(U, y):
    x = np.zeros(len(y))
    n = len(x)-1
    
    x[n] = y[n]/U[n,n]
    n = n-1
    while n >= 0:
        x[n] = (y[n] - (U[n, n+1:] @ x[n+1:])) / U[n, n]
        n = n-1
    
    return x



## Ejercicio 16

In [3]:
# a

# Dada una Matriz A de nxn, me devuelve la descomposicion LU de A (A = LU)
# Asumo que A no es necesario que haga pivoteos
def LU(A):
    L = np.eye(A.shape[0])
    U = A.copy().astype(float)
    
    for i in range(A.shape[0]):
        for j in range(i+1, A.shape[0]):
            L[j,i] =  U[j,i]/U[i,i]
            U[j, i:] = U[j,i:] - (U[j,i]/U[i,i]) * U[i,i:]
            
        
    return  L,U

#b 

def resolverA(A, b):
    L, U = LU(A)
    
    y = res_L(L, b)
    x = res_U(U, y)
    
    return x
    

In [4]:
A = np.array([[1.,3.,0.],[7.,1.,2.],[8.,10.,1.]])
b = np.array([1,2,4])

x = resolverA(A,b)

print('Ax es igual a', A@x, 'que es lo mismo que b')

Ax es igual a [1. 2. 4.] que es lo mismo que b


In [ ]:

A@x

Esto es un branch para practicar

In [ ]:
sym.MatrixSymbol('A', 2, 2)



# Ejercicio 5

In [6]:
# Dada una Matriz L bidiagonal inferior, con unos en la diagonal principal
# resuelve Ly = b

def res_L_reducido(L, b):
    m = len(b)
    y = np.zeros(m)
    y[0] = b[0] # Separo caso 
    for i in range(1, m):
        y[i] = b[i] - L[i, i-1] * y[i-1]
    
    return y

# Dada una Matriz U bidiagonal superior, la afuncion resuelve Ux=y
# def res_U_reducido(U, y):